In [ ]:
def allow_direct_imports_from(dirname):
    import sys
    if dirname not in sys.path:
        sys.path.append(dirname)
        
allow_direct_imports_from('automl/efficientdet')

import os, shutil, matplotlib.pyplot as plt, tensorflow as tf
import tf2.infer_lib as infer_lib, util, custom_callbacks, train_data_generator, dynamicattacker as attacker

In [ ]:
MODEL = 'efficientdet-lite4'

def main(download_model=False):
    log_dir = util.ensure_empty_dir('log_dir')
    gpu = tf.config.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu, True)

    victim_model = util.get_victim_model(MODEL, download_model)
    config_override = {'nms_configs': {'iou_thresh': .5, 'score_thresh': .5},
                       'image_size': 480}
    model = attacker.DynamicPatchAttacker(victim_model,
                                          initial_weights='save_dir/pa',
                                          config_override=config_override,
                                          visualize_freq=200)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3), run_eagerly=False)

    datasets: dict = train_data_generator.partition(model.config, 'downloaded_images', 'labels',
                                                               batch_size=24, shuffle=True)

    train_ds = datasets['train']['dataset']
    val_ds = datasets['val']['dataset']
    train_len = datasets['train']['length']
    val_len = datasets['val']['length']
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir, write_graph=False,
                                                 write_steps_per_second=True,
                                                 update_freq='epoch')
    model.tb = tb_callback

    save_dir = util.ensure_empty_dir('save_dir')
    save_file = 'patch_{epoch:02d}_{val_mean_score_delta:.4f}'
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, steps_per_epoch=train_len,
#                         initial_epoch=12,
                        validation_steps=val_len,
                        callbacks=[tb_callback,
                                   tf.keras.callbacks.ModelCheckpoint(os.path.join(save_dir, save_file),
                                                                      monitor='val_loss',
                                                                      verbose=1,
                                                                      save_best_only=False,
                                                                      save_weights_only=True,
                                                                      mode='auto',
                                                                      save_freq='epoch',
                                                                      options=None,
                                                                      initial_value_threshold=None
                                                                      ),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', verbose=1, factor=.5, min_lr=1e-5)
                                   ])
    
    return hist, model

In [ ]:
hist, model = main()

In [ ]:
def allow_direct_imports_from(dirname):
    import sys
    if dirname not in sys.path:
        sys.path.append(dirname)

allow_direct_imports_from('automl/efficientdet')

import os, shutil, tensorflow as tf
import tf2.infer_lib as infer_lib, util, custom_callbacks, train_data_generator, attack_detection_v2 as defender

In [ ]:
MODEL = 'efficientdet-lite4'

def main(download_model=False):
    log_dir = util.ensure_empty_dir('log_dir')
    gpu = tf.config.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu, True)

    victim_model = util.get_victim_model(MODEL, download_model)
    config_override = {'nms_configs': {'iou_thresh': .5, 'score_thresh': .5},
                       'image_size': 480}
    model = defender.PatchAttackDefender(victim_model,
#                                           initial_weights='save_dir/patch_04_0.5024',
                                         eval_patch_weights='save_dir_rand_print_dropout/patch_100_0.6910',
                                          config_override=config_override,
                                          visualize_freq=50)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), run_eagerly=False)

    datasets: dict = train_data_generator.partition(model.config, 'downloaded_images', 'labels',
                                                               batch_size=64, shuffle=True)

    train_ds = datasets['train']['dataset']
    val_ds = datasets['val']['dataset']
    train_len = datasets['train']['length']
    val_len = datasets['val']['length']
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir, write_graph=False,
                                                 write_steps_per_second=True,
                                                 update_freq='epoch')
    model.tb = tb_callback

    save_dir = util.ensure_empty_dir('save_dir')
    save_file = 'patch_{epoch:02d}_{val_loss:.4f}'
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, steps_per_epoch=train_len,
#                         initial_epoch=12,
                        validation_steps=val_len,
                        callbacks=[tb_callback,
                                   tf.keras.callbacks.ModelCheckpoint(os.path.join(save_dir, save_file),
                                                                      monitor='val_loss',
                                                                      verbose=1,
                                                                      save_best_only=False,
                                                                      save_weights_only=True,
                                                                      mode='auto',
                                                                      save_freq='epoch',
                                                                      options=None,
                                                                      initial_value_threshold=None
                                                                      ),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', verbose=1, factor=.5, min_lr=1e-5)
                                   ])
    
    return hist, model

In [ ]:
hist, model = main()

In [ ]:
def allow_direct_imports_from(dirname):
    import sys
    if dirname not in sys.path:
        sys.path.append(dirname)

allow_direct_imports_from('automl/efficientdet')

import os, shutil, tensorflow as tf
import tf2.infer_lib as infer_lib, util, custom_callbacks, train_data_generator, attack_detection_v2 as defender

In [ ]:
MODEL = 'efficientdet-lite4'

def main(download_model=False):
    log_dir = util.ensure_empty_dir('log_dir')
    gpu = tf.config.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu, True)

    victim_model = util.get_victim_model(MODEL, download_model)
    config_override = {'nms_configs': {'iou_thresh': .5, 'score_thresh': .5},
                       'image_size': 480}
    model = defender.PatchAttackDefender(victim_model,
#                                           initial_weights='save_dir/patch_04_0.5024',
                                         eval_patch_weights='save_dir_rand_print_dropout/patch_100_0.6910',
                                          config_override=config_override,
                                          visualize_freq=50)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), run_eagerly=False)

    datasets: dict = train_data_generator.partition(model.config, 'downloaded_images', 'labels',
                                                               batch_size=64, shuffle=True)

    train_ds = datasets['train']['dataset']
    val_ds = datasets['val']['dataset']
    train_len = datasets['train']['length']
    val_len = datasets['val']['length']
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir, write_graph=False,
                                                 write_steps_per_second=True,
                                                 update_freq='epoch')
    model.tb = tb_callback

    save_dir = util.ensure_empty_dir('save_dir')
    save_file = 'patch_{epoch:02d}_{val_loss:.4f}'
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, steps_per_epoch=train_len,
#                         initial_epoch=12,
                        validation_steps=val_len,
                        callbacks=[tb_callback,
                                   tf.keras.callbacks.ModelCheckpoint(os.path.join(save_dir, save_file),
                                                                      monitor='val_loss',
                                                                      verbose=1,
                                                                      save_best_only=False,
                                                                      save_weights_only=True,
                                                                      mode='auto',
                                                                      save_freq='epoch',
                                                                      options=None,
                                                                      initial_value_threshold=None
                                                                      ),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', verbose=1, factor=.5, min_lr=1e-5)
                                   ])
    
    return hist, model

In [ ]:
hist, model = main()

In [2]:
MODEL = 'efficientdet-lite4'

def main(download_model=False):
    log_dir = util.ensure_empty_dir('log_dir')
    gpu = tf.config.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu, True)

    victim_model = util.get_victim_model(MODEL, download_model)
    config_override = {'nms_configs': {'iou_thresh': .5, 'score_thresh': .5},
                       'image_size': 480}
    model = defender.PatchAttackDefender(victim_model,
#                                           initial_weights='save_dir/patch_04_0.5024',
                                         eval_patch_weights='save_dir_rand_print_dropout/patch_100_0.6910',
                                          config_override=config_override,
                                          visualize_freq=50)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-2), run_eagerly=False)

    datasets: dict = train_data_generator.partition(model.config, 'downloaded_images', 'labels',
                                                               batch_size=64, shuffle=True)

    train_ds = datasets['train']['dataset']
    val_ds = datasets['val']['dataset']
    train_len = datasets['train']['length']
    val_len = datasets['val']['length']
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir, write_graph=False,
                                                 write_steps_per_second=True,
                                                 update_freq='epoch')
    model.tb = tb_callback

    save_dir = util.ensure_empty_dir('save_dir')
    save_file = 'patch_{epoch:02d}_{val_loss:.4f}'
    hist = model.fit(train_ds, validation_data=val_ds, epochs=100, steps_per_epoch=train_len,
#                         initial_epoch=12,
                        validation_steps=val_len,
                        callbacks=[tb_callback,
                                   tf.keras.callbacks.ModelCheckpoint(os.path.join(save_dir, save_file),
                                                                      monitor='val_loss',
                                                                      verbose=1,
                                                                      save_best_only=False,
                                                                      save_weights_only=True,
                                                                      mode='auto',
                                                                      save_freq='epoch',
                                                                      options=None,
                                                                      initial_value_threshold=None
                                                                      ),
                                   tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', verbose=1, factor=.5, min_lr=1e-5)
                                   ])
    
    return hist, model

In [ ]:
hist, model = main()

/tmp/__autograph_generated_filegvy0pzz2.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  ag__.for_stmt(ag__.ld(self).updates, None, loop_body, get_state, set_state, (), {'iterate_names': 'u'})
/phyadv/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:
filtering dataset by label constraints...
done. data size is 64115
training on 51292 images, validating on 6411


Epoch 1/100


Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


802/802 [==============================] - ETA: 0s - loss: 1.8574

/tmp/__autograph_generated_filegvy0pzz2.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  ag__.for_stmt(ag__.ld(self).updates, None, loop_body, get_state, set_state, (), {'iterate_names': 'u'})
/phyadv/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:



Epoch 1: saving model to save_dir/patch_01_1.8400
802/802 [==============================] - 1097s 1s/step - loss: 1.8574 - val_loss: 1.8400 - lr: 0.0100
Epoch 2/100
802/802 [==============================] - ETA: 0s - loss: 0.9391
Epoch 2: saving model to save_dir/patch_02_3.4201
802/802 [==============================] - 1039s 1s/step - loss: 0.9391 - val_loss: 3.4201 - lr: 0.0100
Epoch 3/100
802/802 [==============================] - ETA: 0s - loss: 0.8387
Epoch 3: saving model to save_dir/patch_03_6.1951
802/802 [==============================] - 1038s 1s/step - loss: 0.8387 - val_loss: 6.1951 - lr: 0.0100
Epoch 4/100
802/802 [==============================] - ETA: 0s - loss: 0.7879
Epoch 4: saving model to save_dir/patch_04_22.3647
802/802 [==============================] - 1038s 1s/step - loss: 0.7879 - val_loss: 22.3647 - lr: 0.0100
Epoch 5/100
802/802 [==============================] - ETA: 0s - loss: 0.7568
Epoch 5: saving model to save_dir/patch_05_21.6993
802/802 [=========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



802/802 [==============================] - ETA: 0s - loss: 0.6194
Epoch 26: saving model to save_dir/patch_26_7.9850
802/802 [==============================] - 1039s 1s/step - loss: 0.6194 - val_loss: 7.9850 - lr: 0.0100
Epoch 27/100
802/802 [==============================] - ETA: 0s - loss: 0.6273
Epoch 27: saving model to save_dir/patch_27_11.4455
802/802 [==============================] - 1036s 1s/step - loss: 0.6273 - val_loss: 11.4455 - lr: 0.0100
Epoch 28/100
753/802 [===========================>..] - ETA: 55s - loss: 0.6258
Epoch 28: saving model to save_dir/patch_28_4.3121
802/802 [==============================] - 1037s 1s/step - loss: 0.6248 - val_loss: 4.3121 - lr: 0.0100
Epoch 29/100
525/802 [==================>...........] - ETA: 5:15 - loss: 0.6178